In [22]:
config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'project',
  'raise_on_warnings': True
}

In [23]:
# create a MySQL server connection object
import mysql.connector
mydb = mysql.connector.connect(**config)
my_cursor = mydb.cursor(dictionary=True)

In [24]:
my_cursor.execute(" SELECT table_name FROM information_schema.tables WHERE table_schema ='project' ")
my_result = my_cursor.fetchall()
my_result

[{'table_name': 'imdb'},
 {'table_name': 'movies'},
 {'table_name': 'roles'},
 {'table_name': 'scores'},
 {'table_name': 'users'}]

`Base Table`

In [25]:
query = "select * from imdb limit 5"
my_cursor.execute(query)
for movie_info in my_cursor:
    print(movie_info)


{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Series_Title': 'The Shawshank Redemption', 'Released_Year': 1994, 'Certificate': 'A', 'Runtime': 142, 'Genre': 'Drama', 'IMDB_Rating': 9.3, 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'Meta_score': 80.0, 'Director': 'Frank Darabont', 'Star1': 'Tim Robbins', 'Star2': 'Morgan Freeman', 'Star3': 'Bob Gunton', 'Star4': 'William Sadler', 'No_of_Votes': 2343110, 'Gross': 28341469}
{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR1,0,67,98_AL_.jpg', 'Series_Title': 'The Godfather', 'Released_Year': 1972, 'Certificate': 'A', 'Runtime': 175, 'Genre': 'Crime, Drama', 'IMDB_Rating': 9.2, 'Overview': "An organized crime dynasty's aging patriarch transf

`Normalization`

### Create Tables

#### Create Table 1: `movies` This table stores detailed information about each movie.

In [ ]:
query = \
"""
CREATE TABLE movies(
    movie_id INTEGER NOT NULL AUTO_INCREMENT, -- Unique identifier for each movie
    title VARCHAR(255) NOT NULL, -- Movie title
    poster_link VARCHAR(255) NOT NULL, -- Link to the movie's poster
    released_year INTEGER NOT NULL, -- Year the movie was released
    certificate VARCHAR(10) NOT NULL, -- Certification of the movie (e.g., PG-13, R)
    runtime INTEGER NOT NULL, -- Duration of the movie in minutes
    genre VARCHAR(255) NOT NULL, -- Genre of the movie
    overview TEXT, -- Brief overview or summary of the movie
    gross DECIMAL(15, 2), -- Gross earnings of the movie
    PRIMARY KEY(movie_id) -- Defining movie_id as the primary key
);
"""
my_cursor.execute(query)

#### Create Table 2: `scores` Stores different scores and ratings related to the movies.

In [ ]:
query = \
"""
CREATE TABLE scores (
    score_id INT NOT NULL AUTO_INCREMENT, -- Unique identifier for each score
    movie_id INT NOT NULL, -- References a movie by its movie_id
    imdb_rating DECIMAL(3, 1) CHECK (imdb_rating BETWEEN 1.0 AND 10.0), -- IMDb rating with a constraint
    meta_score INT CHECK (meta_score BETWEEN 0 AND 100), -- Metascore rating with a constraint
    PRIMARY KEY (score_id), -- Defining score_id as the primary key
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id) ON DELETE CASCADE -- Connecting to the movies table, enabling cascade delete
);
"""
my_cursor.execute(query)

#### Create Table 3: `roles` Manages information about the cast and their roles in movies.

In [ ]:
query = \
"""
CREATE TABLE roles (
    role_id INT NOT NULL AUTO_INCREMENT, -- Unique identifier for each role
    movie_id INT NOT NULL, -- References a movie by its movie_id
    actor_name VARCHAR(255) NOT NULL, -- Name of the actor/actress
    role VARCHAR(255) NOT NULL, -- Role played by the actor/actress
    PRIMARY KEY (role_id), -- Defining role_id as the primary key
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id) ON DELETE CASCADE -- Connecting to the movies table, enabling cascade delete
);
"""
my_cursor.execute(query)

#### Create Table 4: `users` Contains user-specific data related to movie reviews and ratings.

In [ ]:
query = \
"""
CREATE TABLE users (
    user_id INT NOT NULL AUTO_INCREMENT, -- Unique identifier for each user
    movie_id INT NOT NULL, -- References a movie by its movie_id
    username VARCHAR(255) NOT NULL, -- Username of the user
    first_name VARCHAR(255), -- First name of the user
    last_name VARCHAR(255), -- Last name of the user
    review VARCHAR(255), -- User's review of the movie
    user_rating DECIMAL(3, 1) NOT NULL CHECK (user_rating BETWEEN 1.0 AND 10.0), -- User's rating with a constraint
    PRIMARY KEY (user_id), -- Defining user_id as the primary key
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id) ON DELETE CASCADE -- Connecting to the movies table, enabling cascade delete
);
"""
my_cursor.execute(query)

### Insert data into the created tables

####  Insert data into 'movies' table

In [ ]:
query = \
"""
INSERT INTO movies (title, poster_link, released_year, certificate, runtime, genre, overview, gross)
SELECT Series_Title, Poster_Link, Released_Year, Certificate, Runtime, Genre, Overview, Gross
FROM imdb;
"""
my_cursor.execute(query)

####  Insert data into 'scores' table

In [ ]:
query = \
"""
INSERT INTO scores (movie_id, imdb_rating, meta_score)
SELECT movie_id, IMDB_rating, Meta_score
FROM imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

####  Insert data into 'roles' for Star1 (actor)

In [ ]:
query = \
"""
INSERT INTO roles (movie_id, actor_name, role)
SELECT movie_id, Star1, "Actor"
from imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

####  Insert data into 'roles' for Star2 (actor)

In [ ]:
query = \
"""
INSERT INTO roles (movie_id, actor_name, role)
SELECT movie_id, Star2, "Actor"
from imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

####  Insert data into 'roles' for Star3 (actor)

In [ ]:
query = \
"""
INSERT INTO roles (movie_id, actor_name, role)
SELECT movie_id, Star3, "Actor"
from imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

####  Insert data into 'roles' for Star4 (actor)

In [ ]:
query = \
"""
INSERT INTO roles (movie_id, actor_name, role)
SELECT movie_id, Star4, "Actor"
from imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

####  Insert data into 'roles' for Director

In [ ]:
query = \
"""
INSERT INTO roles (movie_id, actor_name, role)
SELECT movie_id, Director, "Director"
from imdb AS i
LEFT JOIN movies AS m
on i.Series_Title = m.title and i.Poster_Link = m.poster_link;
"""
my_cursor.execute(query)

## Queries

#### Retrieve top 5 movies based on IMDb ratings

In [42]:
query = \
"""
select 
    m.title, s.imdb_rating 
from movies m
left join scores s
on m.movie_id = s.movie_id
order by s.imdb_rating desc
limit 5
"""
my_cursor.execute(query)
for movie in my_cursor:
    print(movie)

{'title': 'The Shawshank Redemption', 'imdb_rating': Decimal('9.3')}
{'title': 'The Godfather', 'imdb_rating': Decimal('9.2')}
{'title': 'The Dark Knight', 'imdb_rating': Decimal('9.0')}
{'title': 'The Godfather: Part II', 'imdb_rating': Decimal('9.0')}
{'title': '12 Angry Men', 'imdb_rating': Decimal('9.0')}


#### Retrieve top 5 movies with the highest gross earnings


In [33]:
query = \
"""
select 
    title, gross 
from movies 
order by gross desc
limit 5
"""
my_cursor.execute(query)
for author in my_cursor:
    print(author)

{'title': 'Star Wars: Episode VII - The Force Awakens', 'gross': Decimal('936662225.00')}
{'title': 'Avatar', 'gross': Decimal('760507625.00')}
{'title': 'Avengers: Infinity War', 'gross': Decimal('678815482.00')}
{'title': 'Titanic', 'gross': Decimal('659325379.00')}
{'title': 'The Avengers', 'gross': Decimal('623279547.00')}


#### Retrieve the director of the movie

In [41]:
movie_name = 'Avatar'
query = \
f"""
select 
    actor_name
from roles r 
left join movies m
on r.movie_id = m.movie_id
where m.title = '{movie_name}'
and r.role = 'Director'
"""
my_cursor.execute(query)
for author in my_cursor:
    print(author)

{'actor_name': 'James Cameron'}
